In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [137]:
# Read the data
df = pd.read_excel('data\comunidades_archivos_consolidados.xlsx')

In [138]:
# Llenar la comunidad por -1 si no es un grupo conocido
df.fillna(-1, inplace=True)

In [139]:
# Diccionario de las categorías
categorias = {
    0: 'Canasta básica y carnes frías',
    1: 'Granos, atunes y caldos',
    2: 'Productos procesados y dulces',
    3: 'Proteínas animales',
    -1: 'Otras categorías'
}

# Agrupar datos por cedula, mes (el mes tiene que calcularse a partir de la variable fecha) y comunidad contando el total de productos comprados
df_agrupado = df.groupby(['Cedula', df['fecha'].dt.month, df['comunidad'].map(categorias)])['Plu'].nunique().reset_index()

# Ordenar por el mes
df_agrupado.sort_values(by = ['Cedula', 'fecha'], inplace = True)

# Obtener la tabla con la categoría más comprada por cada cedula y mes
df_agrupado_solo_maximo_comunidad = df_agrupado.groupby(["Cedula", "fecha"], as_index = False).apply(lambda x: x.iloc[0])

In [140]:
def obtener_conteo_cruzado(df_agrupado : pd.DataFrame) -> pd.DataFrame:
    # Calcular el porcentaje por categorías
    categorias = df_agrupado['comunidad'].unique()

    df_sept = df_agrupado[df_agrupado['fecha'] == 9]
    df_oct = df_agrupado[df_agrupado['fecha'] == 10]
    conteos_totales = {}

    for categoria in categorias:
        df_temp_categ = df_sept[(df_sept['comunidad'] == categoria)]
        cedulas = df_temp_categ['Cedula'].unique()
        df_oct_ = df_oct[(df_oct['Cedula'].isin(cedulas))]
        total_cedulas = df_oct_['Cedula'].unique().shape[0]

        conteos_totales[categoria] = df_oct_.groupby('comunidad')['Cedula'].nunique() / total_cedulas

    return pd.DataFrame(data = conteos_totales.values(), index = conteos_totales.keys())

In [141]:
obtener_conteo_cruzado(df_agrupado_solo_maximo_comunidad)

comunidad,Canasta básica y carnes frías,"Granos, atunes y caldos",Otras categorías,Productos procesados y dulces,Proteínas animales
Canasta básica y carnes frías,0.910782,0.030457,0.041818,0.009695,0.007247
"Granos, atunes y caldos",0.620818,0.221190,0.100372,0.027881,0.029740
Productos procesados y dulces,0.421296,0.023148,0.226852,0.273148,0.055556
Proteínas animales,0.368852,0.049180,0.229508,0.098361,0.254098
Otras categorías,0.555260,0.066578,0.274301,0.063915,0.039947


# Evolución entre Septiembre y Octubre

In [146]:
df.groupby(df['fecha'].map(lambda x: x.month))['Cedula'].nunique()

fecha
9     12939
10    19084
Name: Cedula, dtype: int64